# Task 1 Reconstruct the Original Meeting Transcripts 

In [1]:
#importing the library
from bs4 import BeautifulSoup as bsoup
import re
import os
import nltk
from __future__ import with_statement
import os.path


Storing the file paths

In [2]:
xml_file_path = "./topics"

In [6]:
xml_file_path_word = "./words"

In [7]:
xml_file_path_segment = "./segments"

#### Function for obtaing the word 

In [9]:
def word_open(w,x,y):
    xfile = os.path.join(xml_file_path_word, w[0]) # joining the path of the word file with the name of the word file
    word_file = open(xfile)
    xmlSoupword = bsoup(word_file,"lxml")
    word_file.close()
    list_word = []     #list to append the words
    check = 0
    word_list = xmlSoupword.find_all(text = False)[3:] 
    if len(y)!= 0:   #checking if there is an end word 
        for wl in word_list: #obtaing the nite:id
            if check == 0 and x[0] == wl.attrs['nite:id']:  #searching for the start word in the nite:id attr
                if wl.text != '':                            #checking if its a vocalsound or anything other than a word
                    list_word.append(wl.text)                #if not true then append it in a list
                seg_return = segmenting(w,[wl.attrs['nite:id']])     #calling the segmenting function
                if seg_return != 0 and  len(list_word) > 0:         #checking if a segment is required and appending 
                    list_word.append(seg_return)
                check = 1
                continue
            elif check == 1 and y[0] not in wl.attrs['nite:id']:      #see if the start word is found and last word isnt found
                if wl.text != '':                                    #again to check if its anything other than a word
                    list_word.append(wl.text)
                seg_return = segmenting(w,[wl.attrs['nite:id']])
                if seg_return != 0 and len(list_word) != 0 and   list_word[-1] != '\n': 
                    list_word.append(seg_return)
                        
            elif check == 1 and y[0] in wl.attrs['nite:id']:     #checking if the last word is found
                if wl.text != '':                                 # appending it to the list of words
                    list_word.append(wl.text)                            
                seg_return = segmenting(w,[wl.attrs['nite:id']])
                if seg_return != 0 and len(list_word) != 0 and  list_word[-1] != '\n':
                    list_word.append(seg_return)
                return list_word                      #returning the list

    elif len(y) == 0:                                   #if there is no end word then this section gets executed
        for wl in word_list:
            if x[0] == wl.attrs['nite:id']:                  #checking if teh start word is present in the attribute of the nite:id
                if wl.text != '':
                    list_word.append(wl.text)
                seg_return = segmenting(w,[wl.attrs['nite:id']])
                if seg_return!= 0 and  len(list_word) > 0:
                    list_word.append(seg_return)
                
                return list_word


In [10]:
def segmenting(w,y):                                     #function for segmenting
    w_file = w[0].replace("words","segments")                  #opening the segment file for a particular
    xfile = os.path.join(xml_file_path_segment, w_file)         
    seg_file = open(xfile)
    xmlSoupseg = bsoup(seg_file,"lxml")
    seg_file.close()
    seg_list = xmlSoupseg.find_all("segment",text=False)
    for s in seg_list:                       #taking each nite child

        nite = s.find("nite:child",href = True, text = False) #reading the nite child
        href_text = nite.get('href')                 #getting the href

        doc_name = re.findall(r'([\w.]+)#',href_text)   #obtaining the word file name start and end segment
        start = re.findall(r'#(?:id)\(([\w.]+)\)',href_text)
        end = re.findall(r'(?:\.\.id)\(([\w.]+)\)',href_text)
        if len(end)!= 0:          #ensuring the end word is present


            if y[0] == end[0]:      #checking the obtained word in the end of the segment if present returns '\n'
                return "\n"


        elif len(end) == 0:        #if there is only one word in segment

            if y[0] == start[0]:   #checking the obtained word in the start word
                return "\n"


    return 0     #if the word isnt an end of a segment returns a 0



In [15]:
save_path = './txt_files'
for xfile in os.listdir(xml_file_path): 
    xfile = os.path.join(xml_file_path, xfile)
    if os.path.isfile(xfile) and xfile.endswith('.xml'):   #opening each file
        
        a = open(xfile)
        xmlSoup = bsoup(a,"lxml")
        a.close()
        star = ["**********\n"]      #two set of stars
        star_space =["\n**********\n"]
        topic = xmlSoup.find_all("topic")       #finding all topic files
        new_list = []                 #list to store the words
        nite_list = []                #temporary list to store the topics read
        for t in topic:                    #obtaing each topics
            nite = t.find_all("nite:child",href = True,text = False)
            word_new_list = []   #list for each sub topics
            if not(set(nite).issubset(set(nite_list))):    #checking if the topic read hasnt been already read
                nite_list.extend(nite)     
                for n in nite:      #obtaing each nite:child of teh topic
                    href_text = n.get('href')
                    doc_name = re.findall(r'([\w.]+)#',href_text)    #obtaining the word file name start and end word
                    start = re.findall(r'#(?:id)\(([\w.]+)\)',href_text)
                    end = re.findall(r'(?:\.\.id)\(([\w.]+)\)',href_text)
                    word_new_list.extend(word_open(doc_name,start,end))    #appending the list of words into a sublist
                if word_new_list[-1] == '\n':       #checking if there is '\n' in the end of list or not
                    word_new_list.extend(star)
                elif  word_new_list[-1] != '\n':
                    word_new_list.extend(star_space)
                new_list.append(word_new_list)
        new_list[-1][-1] = new_list[-1][-1].strip()  #removing the space in the end
        
        # writing the words in a file   
        file_name = re.findall(r'\\(.+).[t]',xfile)
        completeName = os.path.join(save_path, file_name[0] +".txt")
        
        with open(completeName, 'w') as f:
            for _list in new_list:
                string1  = ' '.join([_string for _string in _list])  
                
                f.write(' ' + string1.replace("\n **********","\n**********"))
        
       
        